## Big Data for Biologists: Decoding Genomic Function - Class 13


##  Learning Objectives
***Students should be able to***
 <ol>
    
<li> <a href=#Arithmetic>Use the bedtools closest and bedtools intersect functions be used to identify expressed genes from active promoter and strong enhancer sites.</a></li>
<li> <a href=#Subtract>Use the bedtools subtract command be used to compare the activity of promoters and enhancers across cell types. </a></li>
<li> <a href=#IntersectBed> Use the intersectBed command to extract sequences corresponding to promoter or enhancer regions for a gene from CHiP-seq data </a></li> 
<li> <a href=#BestPractices> Use bedtools best practices to work with large bed files in a memory-efficient manner.</a></li>

## Using the bedtools closest and bedtools intersect functions to identify expressed genes from active promoter and strong enhancer sites. <a name='Arithmetic' />

We obtain a list of promoters and enhancers for the H1-hESC cell line (embryonic stem cells) from ENCODE [here](http://genome.ucsc.edu/cgi-bin/hgFileUi?db=hg19&g=wgEncodeBroadHmm). Both files are in the [Bed6 format](https://genome.ucsc.edu/FAQ/FAQformat#format1). By comparing the location of active promoters and strong enhancers in the genome to the location of genes, we can get a good sense of which genes are active. 

Let's examine the contents of the active promoter and strong enhancer files: 

In [ ]:
# The active promoter file: 
!head -n10 data/wgEncodeBroadHmmH1hescHMM.active_promoters.bed

In [ ]:
#The strong enhancer file: 
!head -n10 data/wgEncodeBroadHmmH1hescHMM.strong_enhancers.bed

We also have a list of gene coordinates for the hg19 human reference genome. The column meanings are as follows: 
* column 1: Chromosome name 
* column 2: Start of transcription 
* column 3: End of transcription 
* column 4: Chromatin state 
* column 5: Place holder (you can ignore this) 
* column 6: Strand information

In [ ]:
!head -n10 data/hg19.gene_coords.bed

We use the **wc** command to determine the total number of genes in the reference genome: 

In [ ]:
!wc -l data/hg19.gene_coords.bed

We would like to see which genes are expressed in the H1-hESC cell type. The [bedtools intersect](http://bedtools.readthedocs.io/en/latest/content/tools/intersect.html) command can be used to intersect the file of active promoters with the list of gene coordinates to determine which genes are being expressed.

In [ ]:
!bedtools intersect -u -a data/hg19.gene_coords.bed  -b data/wgEncodeBroadHmmH1hescHMM.active_promoters.bed  > expressed_genes_H1.active_promoters.bed

Let's examine the resulting file to see which genes intersect active promoters and are therefore turned on in the H1 cell line:

In [ ]:
!head -n10 expressed_genes_H1.active_promoters.bed

We can use the **wc** command to see how many genes are expressed in total: 

In [ ]:
! wc -l expressed_genes_H1.active_promoters.bed 

Looks like there are 10,081 expressed genes in the cell line, which is slightly less than half of all reference genes. 

Now, let's try the same intersection for the strong enhancers: 

In [ ]:
!bedtools intersect -u -a data/hg19.gene_coords.bed -b data/wgEncodeBroadHmmH1hescHMM.strong_enhancers.bed > expressed_genes_H1.strong_enhancers.bed

Let's see how many genes show up as intersecting a strong  enhancer:

In [ ]:
!wc -l expressed_genes_H1.strong_enhancers.bed

Note that we observe a much smaller number of genes -- only 4158 as opposed to 10081 when we examined intersection with active promoters. What could account for this difference? There are two possible explanations. 

Not every expressed gene will be associated with a strong enhancer. Some may be associated with a weak enhancer, or not  have an associated enhancer at all. 

Additionally, many enhancers are distal-acting -- they are located several hundred bases away from the target gene. After a transcription factor has bound to the enhancer region, the DNA must form a loop to bring the transcription factor into contact with the target gene:
![Enhancers are several hundred bases away from target genes](../Images/13_enhancer_position.png)

So we don't expect most of the enhancers to intersect the target gene. However, we expect the enhancer to be fairly close to the target gene. Generally (but not always!), the closest gene to a strong enhancer is that enhancer's target gene. We can then identify expressed genes from our list of strong enhancers by using the [**bedtools closest**](http://bedtools.readthedocs.io/en/latest/content/tools/closest.html) command. 

*closest* searches for overlapping features in A and B. In the event that no feature in B overlaps the current feature in A, closest will report the nearest (that is, least genomic distance from the start or end of A) feature in B. Note that closest will report an overlapping feature as the closest—that is, it does not restrict to closest non-overlapping feature. The following iconic “cheatsheet” summarizes the functionality available through the various options provided by the closest tool.
![bedtools closest cheat sheet](../Images/11_bedtools_closest_cheatsheet.png)


We would like to know how far the enhancer is from the target gene, so we add the -d flag to report this distance. We would also like all genes to be reported in the case of ties, so we use the *-t all* flag. 

In [ ]:
!bedtools closest -d -t all -a data/wgEncodeBroadHmmH1hescHMM.strong_enhancers.bed -b data/hg19.gene_coords.bed > expressed_genes.H1.closest.bed 

Let's examine the output:

In [ ]:
!head -n10 expressed_genes.H1.closest.bed 

Perform a sort operation on the gene name column (column 13) to count the number of unique genes identified by the *bedtools closest* command: 

In [ ]:
#cut column 13 from the bed file (this contains the gene names)
!cut -f13 expressed_genes.H1.closest.bed > tmp1

#sort the gene names 
!sort  tmp1 > tmp2 
#get the unique gene names from the sorted data. 
!uniq tmp2 > tmp3 

#count the number of lines in the file. 
!wc -l tmp3 

Now we see 6975 genes, as opposed to 4158 when we used the bedtools intersect command. 

## Use the bedtools subtract command to compare the activity of promoters and enhancers across cell types. <a name='Subtract' />

How would the gene expression profile change if we examined a different cell type? We have downloaded data for the Hepg cell line (from the liver). We repeat our analysis from above: 

In [ ]:
# Here is the file of active promoter regions in the Hepg2 cell line: 
!head -n10 data/wgEncodeBroadHmmHepg2HMM.active_promoters.bed

In [ ]:
#Intersect the active promoters file with the gene coordinates file to get the list of expressed genes 
# in the Hepg2 cell line 
# BEGIN SOLUTION 
# END SOLUTION 

In [ ]:
#Here is the file of strong enhancers in the Hegp2 cell line: 
!head -n10 data/wgEncodeBroadHmmHepg2HMM.strong_enhancers.bed

In [ ]:
# Use the bedtools closest command to map strong enhancers to active genes  
# BEGIN SOLUTION 
# END SOLUTION 

We are now interested in the different genes that are expressed in the H1 cell line as compared to the Hepg2 cell line. We can use the [bedtools subtract](http://bedtools.readthedocs.io/en/latest/content/tools/subtract.html) command to identify entries that are present in one bed file but not present in another bed file. 

The syntax for this command is: 

bedtools subtract -a **fileA** -b **fileB**

Any region from fileB that overlaps a region in fileA will be subtracted from fileA: 

![bedtools closest cheat sheet](../Images/12_subtract.png)




In [ ]:
#Subtract the promoter-intersected genes of the H1 cell line from 
#the promoter-intersected genes of the Hepg2 cell line. 
## BEGIN SOLUTION 
## END SOLUTION 

In [ ]:
#Subtract the promoter-interested genes of the Hepg2 cell line from 
#the promoter-intersected genes of the H1 cell line.
## BEGIN SOLUTION 
## END SOLUTION 

## Use the intersectBed command to extract sequences corresponding to promoter or enhancer regions for a gene from ChIP-seq data <a name='IntersectBed'>

We have fetched a list of enhancer-like and promoter-like regions for the H1 embryonic stem cell line in humans (hg19)

These are stored in the following files: 

**ENCSR000DRY_predictions_promoter.bed** for the promoter file.

**ENCSR000ANP_predictions_enhancer.bed** for the enhnacer file. 

In [ ]:
!head data/ENCSR000DRY_predictions_promoter.bed

#### What file format is the data in? What is contained in each of the first five columns? 
Your answer: 
    
#### Bonus question -- can you explain the meaning of the remaining columns in the file (column 6 and up)? 
Your answer:

Prior studies have shown that the gene **LIN28A** is associated with cell differentiation, and you hypothesize that this gene is likely to be expressed in the H1 cell line, as these are embryonic cells that are undergoing differentiation.  Look up LIN28A in the [WashU Genome Browser](http://epigenomegateway.wustl.edu/browser/) to find the chromosome and position of the LIN28A gene.  Fill in the chromosome and the starting and ending coordinates of LIN28A below:

In [ ]:
chrom_lin28a="chr1" #replace "" with the chromosome of the gene 
startpos_lin28a=26737268 #replace None with the gene start position 
endpos_lin28a=26756213 #replace None with the gene end position

We will use the bedtools intersect command to find the H1  promoter and enhancer peaks that are within 50kb of the LIN28A gene. 

In [ ]:
#You need to store LIN28A's coordinates in a bed file in order to run the intersect command 
## Execute this code block to generate a bed file containing the LIN28A coordinates, including regions 50k 
## upstream & downstream of the gene 

#add 50kb to the start and end coordinates 
startpos_lin28a-=50000
endpos_lin28a+=50000

outf=open("LIN28A.coords.bed",'w')

#convert coordinates to string values in preparation for writing to output file
output_line=[str(i) for i in [chrom_lin28a,startpos_lin28a,endpos_lin28a,'lin28a'] ]

#write the coordinates to the output file. 
outf.write('\t'.join(output_line)+'\n')

#close the output file.
outf.close() 

In [ ]:
!cat LIN28A.coords.bed

In [ ]:
## Now you can intersect the promoter files with the LIN28A coordinates 

## promoter peaks 
!bedtools intersect -wa -a data/ENCSR000DRY_predictions_promoter.bed -b LIN28A.coords.bed 

print()

## you can use the intersectBed shortcut, it will do the same thing 
!intersectBed -wa -a data/ENCSR000DRY_predictions_promoter.bed -b LIN28A.coords.bed

print()

##TODO: write a command to intersect the enhancer peak files with LIN28A coordinates 
## BEGIN SOLUTION 
## END SOLUTION 

## Bedtools best practices  <a name='BestPractices' />

In your course project, you will find yourself working with large files, such as a file containing all known transcription factor locations in the genome. There are two things we recommend to make bedtools operations on large files faster and more memory efficient: 

**1) Whenever a BEDTool compares two files of features, the “B” file is loaded into memory. By contrast, the “A” file is processed line by line and compared with the features from B. Therefore to minimize memory usage, one should set the smaller of the two files as the B file.**

In [ ]:
%%timeit
!bedtools intersect -wa -a data/ENCSR000DRY_predictions_promoter.bed -b LIN28A.coords.bed 

In [ ]:
%%timeit
!bedtools intersect -wa -b data/ENCSR000DRY_predictions_promoter.bed -a LIN28A.coords.bed 

Notice how the second command takes longer to execute. The difference is small for our example, but will quickly increase for much larger files. 

**2) Using the -sorted option on pre-sorted a & b files will invoke a memory-efficient intersection algorithm and avoid the default sorting of the files on-the-fly.**



In [ ]:
#sort our files 
!bedtools sort -i data/ENCSR000DRY_predictions_promoter.bed > ENCSR000DRY_predictions_promoter.sorted.bed 

In [ ]:
%%timeit

#run bedtools intersect without the sorted flag 
!bedtools intersect -wa -a ENCSR000DRY_predictions_promoter.sorted.bed -b LIN28A.coords.bed 

In [ ]:
%%timeit

#run bedtools intersect with the -sorted flag 
!bedtools intersect -sorted -wa -a ENCSR000DRY_predictions_promoter.sorted.bed -b LIN28A.coords.bed 

The genomewide motif file we will use for the course project is "/data/project/motifs.bed.gz". You can experiment with the commands below to see if you get a speedup with the two suggestions above. 
Warning: Using the non-optimized bedtools intersect command may crash  your kernel, so *MAKE SURE YOU SAVE YOUR WORK* before running the below commands. 

In [ ]:
%%time
#non-optimized 
!bedtools intersect -a LIN28A.coords.bed -b /data/project/motifs.bed.gz > tmp


In [ ]:
%%time 
#using the smaller file as file -b 
!bedtools intersect -b LIN28A.coords.bed -a /data/project/motifs.bed.gz > tmp

In [ ]:
%%time 
#adding the sorted flag 
!bedtools intersect -sorted -b LIN28A.coords.bed -a /data/project/motifs.bed.gz > tmp